# Hackathon SPE 2020 | Archivo de lectura

<br>**Presentado por:** <br>
- Geoc. Juan Camilo Burgos Flórez
- Geoc. Luis Fernando Chacón 
- Geoc. Ing. Carlos Fabián Sánchez Suárez

-----

### Desarrollo

<br> Para el desarrollo de esta actividad se contaron con datos oficiales de la ANH de reportes de producción de crudo y se trabajó para los años 2018, 2019 y 2020. Adicionalmente, con archivos encriptados "Blind Test" que se tendrán en cuenta en esta actividad. <br>
<br>El archivo de lectura contiene el código que realiza las operaciones de obtención de la información, limpia los archivos, y los convierte en formato long. Posteriormente, crea una base de datos en SQLite para facilitar el posterior manejo de datos y búsqueda de información.<br>

-----

In [1]:
# Importación de librerías

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3

In [2]:
# URL de los datos oficiales que se van a trabajar

url_prod_2018 = "http://www.anh.gov.co/Operaciones-Regal%c3%adas-y-Participaciones/Sistema-Integrado-de-Operaciones/Documents/Producci%c3%b3n%20Fiscalizada%20Crudo%202018.xlsx"
url_prod_2019 = "http://www.anh.gov.co/Operaciones-Regal%c3%adas-y-Participaciones/Sistema-Integrado-de-Operaciones/Documents/Producci%c3%b3n%20Fiscalizada%20Crudo%202019-DIC.xlsx"
url_prod_2020 = "http://www.anh.gov.co/Operaciones-Regal%c3%adas-y-Participaciones/Sistema-Integrado-de-Operaciones/Documentos%20compartidos/Producci%c3%b3n%20Fiscalizada%20Crudo%202020%20Agosto.xlsx"

In [3]:
# Funcion que descarga, almaceno y limpia los datos

def get_data(data, year):
    from_url =  pd.read_excel(data)                  # Carga el archivo de excel a una variable temporal 
    for i in range(len(from_url)):                   # Encuentra los nombres de las columnas y reasigna el encabezado
        if from_url.iloc[i,0] == "Departamento":          # A partir de la palabra "Departamento" en la primera columna 
            num = i
            break
    new_df = from_url.iloc[num:].reset_index(drop=True)
    encabezado = new_df.iloc[0].str.capitalize()     # Estandariza los nombres del encabezado
    new_df = new_df.rename(columns=encabezado)
    new_df = new_df.iloc[1:].reset_index(drop=True)
    new_df = new_df.dropna()
    new_df["Año"] = year                             # Crea una nueva columna con el año al que corresponden los datos
    
    return new_df

In [4]:
# Obtiene los archivos y almacena la información en un data frame para cada año

df_2018 = get_data (url_prod_2018, 2018)
df_2019 = get_data (url_prod_2019, 2019)
df_2020 = get_data (url_prod_2020, 2020)

df_2020.head()                    # Ejemplo

,Departamento,Municipio,Operadora,Contrato,Campo,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Año
0,ANTIOQUIA,PUERTO NARE,ECOPETROL S.A.,TECA COCORNA,AREA TECA-COCORNA,1765.84,1689.32,1826.93,1537.57,1238.72,1049.21,975.58,949,2020
1,ANTIOQUIA,PUERTO TRIUNFO,ECOPETROL S.A.,TECA COCORNA,AREA TECA-COCORNA,119.7,118.32,81.96,0,0,0,14.05,104.08,2020
2,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE,11636.6,11584.2,11488.2,10711.5,9483.4,9402.73,9599.38,9527.73,2020
3,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE SUR,1852.6,1763.47,1831.5,1838.38,1539.16,1541.67,1573.16,1514.94,2020
4,ANTIOQUIA,PUERTO NARE,MANSAROVAR ENERGY COLOMBIA LTD,NARE,NARE SUR,187.33,184.97,173.58,191.32,193.94,166.74,168.84,152.02,2020


In [5]:
# Guarda los archivos en un solo data frame para facilidad de manejo

base_datos = pd.concat([df_2018,df_2019,df_2020]).reset_index(drop=True)
base_datos = base_datos.infer_objects()
base_datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Departamento  1338 non-null   object 
 1   Municipio     1338 non-null   object 
 2   Operadora     1338 non-null   object 
 3   Contrato      1338 non-null   object 
 4   Campo         1338 non-null   object 
 5   Enero         1338 non-null   float64
 6   Febrero       1338 non-null   float64
 7   Marzo         1338 non-null   float64
 8   Abril         1338 non-null   float64
 9   Mayo          1338 non-null   float64
 10  Junio         1338 non-null   float64
 11  Julio         1338 non-null   float64
 12  Agosto        1338 non-null   float64
 13  Septiembre    925 non-null    float64
 14  Octubre       925 non-null    float64
 15  Noviembre     925 non-null    float64
 16  Diciembre     925 non-null    float64
 17  Año           1338 non-null   int64  
dtypes: float64(12), int64(1), ob

In [6]:
# Funcion que transforma la data en formato LONG

def long(data):
    melted = pd.melt(data, id_vars=["Departamento","Municipio","Operadora", "Contrato", "Campo","Año"],\
                    value_vars = ["Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio","Agosto","Septiembre",\
                                  "Octubre","Noviembre","Diciembre"],\
                    var_name = "Mes",\
                    value_name ="Produccion")
    return melted

In [7]:
# Nuevos datos en formato LONG, que será nuestra tabla de la base de datos

base_datos = long(base_datos)
base_datos                    

,Departamento,Municipio,Operadora,Contrato,Campo,Año,Mes,Produccion
0,ANTIOQUIA,PUERTO NARE,ECOPETROL S.A.,OPERACION-DIRECTA ECOPETROL,AREA TECA-COCORNA,2018,Enero,1290.888387
1,ANTIOQUIA,PUERTO NARE,MANSAROVAR ENERGY COLOMBIA LTD,NARE,NARE SUR,2018,Enero,219.071290
2,ANTIOQUIA,PUERTO NARE,MANSAROVAR ENERGY COLOMBIA LTD,NARE,UNDERRIVER,2018,Enero,800.340323
3,ANTIOQUIA,PUERTO TRIUNFO,ECOPETROL S.A.,OPERACION-DIRECTA ECOPETROL,AREA TECA-COCORNA,2018,Enero,289.360000
4,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE,2018,Enero,11363.255806
...,...,...,...,...,...,...,...,...
16051,TOLIMA,CHAPARRAL,HOCOL S.A.,TOLIMA,RÍO SALDAÑA,2020,Diciembre,NaN
16052,TOLIMA,ORTEGA,HOCOL S.A.,TOLDADO,TOLDADO,2020,Diciembre,NaN
16053,TOLIMA,PIEDRAS,HOCOL S.A.,PULI,TOQUI TOQUI,2020,Diciembre,NaN
16054,TOLIMA,ALVARADO,HOCOL S.A.,ARMERO,TOTARE,2020,Diciembre,NaN


In [8]:
# Creación de la base de datos en SQLite

engine = create_engine("sqlite:///HackathonSPE.db")
sqlite_connection = engine.connect()

# Crea la tabla "TablaProduccion" en la base de datos, a partir de la información de la variable con los datos en formato long
base_datos.to_sql("TablaProduccion", sqlite_connection, if_exists ="replace")     

-----


**VERIFICACIÓN**

In [9]:
# Cargar datos a data frame a partir de la base de datos creada
# Para verificar que se haya creado correctamente la base de datos y que su contenido corresponda 


pd.options.display.float_format = '{:.2f}'.format       # Formato de dos decimales en floats

cnx = sqlite3.connect('HackathonSPE.db')
df_produccion = pd.read_sql_query("SELECT * FROM TablaProduccion", cnx, index_col = "index")

df_produccion

,Departamento,Municipio,Operadora,Contrato,Campo,Año,Mes,Produccion
index,,,,,,,,
0,ANTIOQUIA,PUERTO NARE,ECOPETROL S.A.,OPERACION-DIRECTA ECOPETROL,AREA TECA-COCORNA,2018,Enero,1290.89
1,ANTIOQUIA,PUERTO NARE,MANSAROVAR ENERGY COLOMBIA LTD,NARE,NARE SUR,2018,Enero,219.07
2,ANTIOQUIA,PUERTO NARE,MANSAROVAR ENERGY COLOMBIA LTD,NARE,UNDERRIVER,2018,Enero,800.34
3,ANTIOQUIA,PUERTO TRIUNFO,ECOPETROL S.A.,OPERACION-DIRECTA ECOPETROL,AREA TECA-COCORNA,2018,Enero,289.36
4,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE,2018,Enero,11363.26
...,...,...,...,...,...,...,...,...
16051,TOLIMA,CHAPARRAL,HOCOL S.A.,TOLIMA,RÍO SALDAÑA,2020,Diciembre,nan
16052,TOLIMA,ORTEGA,HOCOL S.A.,TOLDADO,TOLDADO,2020,Diciembre,nan
16053,TOLIMA,PIEDRAS,HOCOL S.A.,PULI,TOQUI TOQUI,2020,Diciembre,nan
